In [9]:
import numpy as np
from pprint import pprint

data = np.genfromtxt('/home/weidong/Downloads/data_2024_18_04_11_45_08.csv', delimiter=',', skip_header=1)


timestep,Id,role_name,Location_x,Location_y,Location_z,Rocation_pitch,Rocation_yaw,Rocation_roll,
Angular_velocity_x,Angular_velocity_y,Angular_velocity_z,Velocity_x,Velocity_y,Velocity_z,
Acceleration_x,Acceleration_y,Acceleration_z,Distance2Ego


In [14]:
pprint(data[:,])  

array([ 11.12124634,  11.12124634,  11.12124634, ..., 111.27124783,
       111.27124783, 111.27124783])
